In [11]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
import pandas as pd

# 加载数据集
df = pd.read_csv(
    "http://labfile.oss.aliyuncs.com/courses/1211/car.data", header=None)
# 设置列名
df.columns = ['buying', 'maint', 'doors',
              'persons', 'lug_boot', 'safety', 'class']
df.head()

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [3]:
from sklearn.model_selection import train_test_split

X = df.iloc[:,:-1]
y = df[['class']]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1382, 6), (346, 6), (1382, 1), (346, 1))

In [4]:
X_train = pd.get_dummies(X_train).values
X_test = pd.get_dummies(X_test).values
y_train = pd.get_dummies(y_train).values
y_test = pd.get_dummies(y_test).values
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1382, 21), (346, 21), (1382, 4), (346, 4))

In [5]:
import tensorflow as tf

def fully_connected(inputs, weights, biases):
    """
    inputs -- 输入 Variable
    weights -- 权重项 Variable
    biases -- 截距项 Variable
    """
    layer = tf.add(tf.matmul(inputs, weights), biases)  # 输入 x 权重 + 截距
    output = tf.nn.relu(layer)  # RELU 激活
    return output

In [6]:
x = tf.placeholder(tf.float32, [None, 21])  # 输入特征张量占位符

# 全连接层 1
W1 = tf.Variable(tf.random.uniform([21, 15]))  # 随机初始化权重
b1 = tf.Variable(tf.random.uniform([15]))
fc1 = fully_connected(x, W1, b1)

# 全连接层 2
W2 = tf.Variable(tf.random.uniform([15, 4]))
b2 = tf.Variable(tf.random.uniform([4]))
outs = fully_connected(fc1, W2, b2)

outs  # 输出

Instructions for updating:
Colocations handled automatically by placer.


<tf.Tensor 'Relu_1:0' shape=(?, 4) dtype=float32>

In [7]:
## 定义损失函数

y = tf.placeholder(tf.float32, [None, 4])  # 真实值标签占位符

# 交叉熵损失函数，reduce_mean 的目的是对每个样本的计算结果求平均
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(logits=outs, labels=y))
loss

<tf.Tensor 'Mean:0' shape=() dtype=float32>

In [8]:
## 定义激活函数

train_step = tf.train.AdamOptimizer(0.01).minimize(loss)
train_step

<tf.Operation 'Adam' type=NoOp>

In [16]:
acc = tf.reduce_mean(tf.cast(tf.math.in_top_k(
    outs, tf.math.argmax(y, 1), k=1), tf.float32))  # 准确率计算

In [17]:
iterations = 1000
feed_dict_train = {x: X_train, y: y_train}
feed_dict_test = {x: X_test, y: y_test}
init = tf.global_variables_initializer()  # 初始化全局变量

with tf.Session() as sess:
    sess.run(init)
    for i in range(iterations):
        if (i+1) % 100 == 0:  # 每间隔 100 次打印 loss 值
            print("Iters [{}/{}], Train Acc [{:.3f}], Test Acc [{:.3f}]".format(
                i+1, iterations, acc.eval(feed_dict=feed_dict_train), acc.eval(feed_dict=feed_dict_test)))
        sess.run(train_step, feed_dict=feed_dict_train)
                
                 

Iters [100/1000], Train Acc [0.886], Test Acc [0.882]
Iters [200/1000], Train Acc [0.927], Test Acc [0.910]
Iters [300/1000], Train Acc [0.949], Test Acc [0.928]
Iters [400/1000], Train Acc [0.958], Test Acc [0.931]
Iters [500/1000], Train Acc [0.959], Test Acc [0.951]
Iters [600/1000], Train Acc [0.958], Test Acc [0.957]
Iters [700/1000], Train Acc [0.959], Test Acc [0.957]
Iters [800/1000], Train Acc [0.963], Test Acc [0.957]
Iters [900/1000], Train Acc [0.964], Test Acc [0.954]
Iters [1000/1000], Train Acc [0.965], Test Acc [0.957]
